In [1]:
import time
import tarfile
import pandas as pd
import io


In [2]:
# Suppress warning for xlrd
import warnings
# Filter all warning that start with "WARNING"
warnings.filterwarnings('ignore')

In [4]:
!mkdir sfdata

In [5]:
#file = '/Users/dirkocoetsee/Downloads/fuse-binaries-dec2014.tar.gz'
file = 'experiments/results/filtered.tar.gz'
tar_stream = tarfile.open(file, mode='r|*')

vocab = {}
words_member = []
words = []
t0 = time.time()
total_time_open = 0
icounter = 0
total_counter = 0
df_list = []
for member_number, member in enumerate(tar_stream):
    t0_loop = time.time()
    if icounter > 1_000_001:
        break
    if not member.isfile():
        continue
    try:
        buffer = tar_stream.extractfile(member)
        dfs = pd.read_excel(io.BytesIO(buffer.read()), sheet_name=None, header=None, index_col=None, nrows=100)
    except Exception as e:
        continue
    for dfi, df in enumerate(dfs.values()):
        # Iterate over all cells
        vals = {str(v)[:20] for v in df.values[:100, :100].flatten()}
        # Update vocab
        vocab_size = len(vocab)
        # Add new words to vocab
        vocab.update({v: vocab_size + i + 1 for i, v in
                      enumerate(val[:20] if isinstance(val, str) else val for val in vals if val not in vocab)})
        # Map df to vocab - all ints
        df_mapped = df[:, :100].map(lambda x: vocab[x] if x in vocab else 0)
        df_mapped['f'] = member.name
        df_mapped['s'] = dfi

        # Add to list
        df_list.append(df_mapped)

        icounter += 1

        # Accumulate sheets and write to parquet
        if len(df_list) >= 1000:
            dfc = pd.concat(df_list)
            dfc.columns = dfc.columns.astype(str)
            print(f'{icounter}', f'{total_counter}', '  ', len(dfs), len(vocab), dfc.shape)

            dfc.to_parquet(f'sfdata/df_{total_counter:04d}.parquet', compression='zstd')
            df_list = []
            total_counter += 1


1000 0    3 119052 (45646, 117)
2000 1    3 223194 (50551, 258)
3000 2    1 316852 (45380, 216)
4000 3    1 388517 (48376, 253)
5000 4    1 452028 (47941, 117)
6000 5    55 547117 (47599, 16386)
7000 6    12 622429 (45055, 404)
8000 7    1 679367 (43394, 350)
9000 8    1 747868 (42250, 258)
10000 9    3 804043 (46973, 149)
11000 10    2 860935 (43895, 258)
12000 11    3 919014 (45429, 766)
13000 12    3 967490 (43491, 258)
14000 13    5 1026174 (47472, 329)
15000 14    1 1084276 (49993, 253)
16000 15    1 1138401 (45833, 138)
17000 16    64 1205638 (49075, 16386)
18000 17    15 1265087 (44713, 316)
19000 18    3 1316135 (46546, 91)
20000 19    1 1368159 (45737, 192)
21000 20    1 1434733 (45030, 151)
22000 21    82 1495680 (47673, 312)
23000 22    6 1547988 (48548, 16386)
24000 23    16 1599898 (45268, 258)
25000 24    3 1648327 (45475, 139)
26000 25    1 1692387 (45406, 16385)
27000 26    3 1733597 (46907, 147)
28000 27    2 1782338 (40210, 95)
29000 28    7 1826140 (45418, 147)
30000

In [ ]:
# Save last batch
dfc = pd.concat(df_list)
dfc.columns = dfc.columns.astype(str)
print(f'{icounter}', f'{total_counter}', '  ', len(dfs), len(vocab), dfc.shape)
dfc.to_parquet(f'sfdata/df_{total_counter:04d}.parquet', compression='zstd')

In [6]:
# Save vocab
vocab_df = pd.DataFrame(vocab.items(), columns=['word', 'id'])
vocab_df.to_parquet('sfdata/vocab.parquet', compression='zstd')

In [7]:
!ls -lah sfdata

total 628952
drwxr-xr-x@ 282 dirkocoetsee  staff   8.8K Feb 27 07:04 .
drwxr-xr-x@  15 dirkocoetsee  staff   480B Feb 27 06:56 ..
-rw-r--r--@   1 dirkocoetsee  staff   255K Feb 27 06:23 df_0000.parquet
-rw-r--r--@   1 dirkocoetsee  staff   310K Feb 27 06:23 df_0001.parquet
-rw-r--r--@   1 dirkocoetsee  staff   308K Feb 27 06:23 df_0002.parquet
-rw-r--r--@   1 dirkocoetsee  staff   291K Feb 27 06:23 df_0003.parquet
-rw-r--r--@   1 dirkocoetsee  staff   250K Feb 27 06:23 df_0004.parquet
-rw-r--r--@   1 dirkocoetsee  staff   4.5M Feb 27 06:24 df_0005.parquet
-rw-r--r--@   1 dirkocoetsee  staff   363K Feb 27 06:24 df_0006.parquet
-rw-r--r--@   1 dirkocoetsee  staff   330K Feb 27 06:24 df_0007.parquet
-rw-r--r--@   1 dirkocoetsee  staff   293K Feb 27 06:24 df_0008.parquet
-rw-r--r--@   1 dirkocoetsee  staff   278K Feb 27 06:24 df_0009.parquet
-rw-r--r--@   1 dirkocoetsee  staff   282K Feb 27 06:24 df_0010.parquet
-rw-r--r--@   1 dirkocoetsee  staff   428K Feb 27 06:25 df_0011.parquet
-rw-r-

In [38]:
!ls -lah sdata

total 2880
drwxr-xr-x@ 13 dirkocoetsee  staff   416B Feb 16 17:31 .
drwxr-xr-x@ 14 dirkocoetsee  staff   448B Feb 16 17:32 ..
-rw-r--r--@  1 dirkocoetsee  staff   681K Feb 16 17:32 df_0000.parquet
-rw-r--r--@  1 dirkocoetsee  staff   100K Feb 16 17:27 df_0001.parquet
-rw-r--r--@  1 dirkocoetsee  staff    63K Feb 16 17:27 df_0002.parquet
-rw-r--r--@  1 dirkocoetsee  staff    68K Feb 16 17:27 df_0003.parquet
-rw-r--r--@  1 dirkocoetsee  staff    72K Feb 16 17:27 df_0004.parquet
-rw-r--r--@  1 dirkocoetsee  staff    66K Feb 16 17:27 df_0005.parquet
-rw-r--r--@  1 dirkocoetsee  staff    57K Feb 16 17:27 df_0006.parquet
-rw-r--r--@  1 dirkocoetsee  staff    76K Feb 16 17:27 df_0007.parquet
-rw-r--r--@  1 dirkocoetsee  staff    76K Feb 16 17:27 df_0008.parquet
-rw-r--r--@  1 dirkocoetsee  staff    58K Feb 16 17:28 df_0009.parquet
-rw-r--r--@  1 dirkocoetsee  staff    20K Feb 16 17:31 vocab.parquet


In [8]:
import duckdb


In [9]:
res = duckdb.sql("select count(*) from 'sfdata/*.parquet'")
res.show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     21393957 │
└──────────────┘



In [10]:
res = duckdb.sql("""
    select distinct f, s from 'sfdata/df*.parquet'
    limit 10
""")
res.show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────────────────────────────────┬───────┐
│                        f                         │   s   │
│                     varchar                      │ int64 │
├──────────────────────────────────────────────────┼───────┤
│ cc-binaries/063545ab-96d0-4a3a-a1b2-f66a8fe21c61 │     1 │
│ cc-binaries/9f8c896a-5ead-40f3-8663-37b338eb1e93 │     0 │
│ cc-binaries/cc6693f3-1e50-4d70-84f7-cc60d3463495 │     0 │
│ cc-binaries/0a379928-a7f3-47bb-908f-f8e3939b92db │     4 │
│ cc-binaries/0a379928-a7f3-47bb-908f-f8e3939b92db │    23 │
│ cc-binaries/0a379928-a7f3-47bb-908f-f8e3939b92db │    24 │
│ cc-binaries/10b7489d-7d42-4628-b1fd-4326405a3637 │     5 │
│ cc-binaries/00455cfa-e1a7-4a2d-ac68-30d2f1dd2bb3 │     0 │
│ cc-binaries/c6dc3d01-98ef-412e-9b22-2c3ea0debe72 │     0 │
│ cc-binaries/73b737c2-9ec9-47d0-a4da-2ecfab3521f5 │     0 │
├──────────────────────────────────────────────────┴───────┤
│ 10 rows                                        2 columns │
└───────────────────────

In [18]:
res = duckdb.sql("""
with sdata as (
    select f, s, "0", "1", "2", "3", "4", "5", "6", "7", "8", "9" from read_parquet('sfdata/df*.parquet', union_by_name=True)
), top_sheet as (
    select distinct f, s from sdata
    limit 2
)
select * from sdata join top_sheet using (f, s)
limit 10
""")
res.show()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────────────────────────────────┬───────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬──────────┬────────┬──────────┐
│                        f                         │   s   │   0    │   1    │   2    │   3    │   4    │   5    │   6    │    7     │   8    │    9     │
│                     varchar                      │ int64 │ double │ double │ double │ double │ double │ double │ double │  double  │ double │  double  │
├──────────────────────────────────────────────────┼───────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼──────────┼────────┼──────────┤
│ cc-binaries/610f8657-cdc2-4c81-89e2-91e659c54e3f │     0 │    0.0 │    0.0 │    0.0 │    0.0 │    0.0 │    0.0 │    0.0 │      0.0 │    0.0 │      0.0 │
│ cc-binaries/610f8657-cdc2-4c81-89e2-91e659c54e3f │     0 │    0.0 │    0.0 │    0.0 │    0.0 │    0.0 │    0.0 │    0.0 │      0.0 │    0.0 │      0.0 │
│ cc-binaries/610f8657-cdc2-4c81-89e2-91e659c54e3f │     0 │    0.0 │ 

In [15]:
res = duckdb.sql("""
    with wscore as (
        select *, case when score > 0 then 1 else 0 end as scorel from 'experiments/results/ftripples.parquet'
    )
    select scorel, count(*) from wscore group by scorel
    limit 10
""")
res.show()

┌────────┬──────────────┐
│ scorel │ count_star() │
│ int32  │    int64     │
├────────┼──────────────┤
│      0 │     52589245 │
│      1 │       553565 │
└────────┴──────────────┘



In [17]:

res = duckdb.sql("""
    with wscore as (
        select *, case when score > 0 then 1 else 0 end as scorel from 'experiments/results/ftripples.parquet'
    )
    select * from wscore
    limit 10
""")
res.show()

┌────────┬────────┬────────┬─────────────────────┬───────┬───────┬───────┬───────┬────────┬────────┬────────┐
│   i    │   j    │   k    │        score        │ sa1i  │ sa2i  │ inter │ denom │ denom1 │ denom2 │ scorel │
│ int64  │ int32  │ int32  │       double        │ int64 │ int64 │ int64 │ int64 │ int64  │ int64  │ int32  │
├────────┼────────┼────────┼─────────────────────┼───────┼───────┼───────┼───────┼────────┼────────┼────────┤
│ 129896 │ 252145 │ 238757 │ 0.23817812022131174 │   115 │    69 │    79 │   274 │    199 │    197 │      1 │
│  98551 │ 172408 │ 270362 │ 0.11230335926631886 │    32 │    35 │    15 │   198 │    241 │    328 │      1 │
│  98551 │ 275402 │ 172408 │ 0.11053469189262151 │    35 │    31 │    16 │   198 │    330 │    241 │      1 │
│  98551 │ 172408 │ 116664 │ 0.10894418174998609 │    31 │    34 │    16 │   198 │    241 │    331 │      1 │
│ 218328 │ 218329 │ 218327 │ 0.10693342094862127 │   194 │    11 │    30 │   332 │    308 │    380 │      1 │
│  22973 │

In [2]:
import pandas as pd
df = pd.read_parquet('experiments/results/parquet2/df_0000.parquet')
print(df.shape)
df.head()

(37506, 106)


,i,0,1,2,3,4,5,6,7,8,...,95,96,97,98,99,f,s,fs,rows,cols
index,,,,,,,,,,,,,,,,,,,,,
0,0,7,14,9,8,2,12,6,10,3,...,1,1,1,1,1,cc-binaries/69f8dc77-cec7-4a92-8519-fb49aab6ae43,0,26,1,13
0,0,7,14,9,8,2,12,6,10,3,...,1,1,1,1,1,cc-binaries/cc8b432c-5517-4358-9162-43af7e5defc5,0,57,1,13
0,0,729,1561,3449,3449,3449,3449,3449,3449,3449,...,3449,3449,3449,3449,3449,cc-binaries/38405202-0ab1-44d3-820e-a52102e1d208,0,87,100,58
1,1,3449,3449,3449,3449,3449,3449,3449,3449,3449,...,3449,3449,3449,3449,3449,cc-binaries/38405202-0ab1-44d3-820e-a52102e1d208,0,87,100,58
2,2,213,1070,1611,3222,2660,4130,1198,283,2475,...,3449,3449,3449,3449,3449,cc-binaries/38405202-0ab1-44d3-820e-a52102e1d208,0,87,100,58


In [17]:
cols = [f'{i}' for i in range(100)]
dfp = df.set_index(['i', 'fs'])[cols].stack().reset_index().rename(columns={'level_2': 'j', 0: 'v'})
dfp[['fs', 'i', 'j', 'v']]
dfp.shape, dfp[dfp.v != 3449].shape

((3750600, 4), (357792, 4))

In [19]:
dfp.head()

,i,fs,j,v
0,0,26,0,7
1,0,26,1,14
2,0,26,2,9
3,0,26,3,8
4,0,26,4,2


In [23]:
# set all the dtypes as int64 inplace
dfp = dfp.astype({'i': 'int64', 'fs': 'int64', 'j': 'int64', 'v': 'int64'})
dfp.dtypes

i     int64
fs    int64
j     int64
v     int64
dtype: object

In [14]:
vocab = pd.read_parquet('experiments/results/parquet2/vocab.parquet')
vocab.head()

,word,id
0,NATIONALITY,2
1,BIKE,3
2,RUN,4
3,POSITION,5
4,SWIM,6


In [16]:
vocab[vocab.id == 3449]

,word,id
3447,nan,3449


In [24]:
vocab.id.max()

np.int64(1603275)

In [37]:
import numpy as np
np.int64(1603275).astype('uint32')

np.uint16(30411)

In [34]:
# Max that int8 can represent
print(np.iinfo(np.int8).max)
# Max that uint8 can represent
print(np.iinfo(np.uint8).max)

127
255


In [31]:
import duckdb
res = duckdb.sql("""
    select max(fs)
    from 'experiments/results/parquet2/df_*.parquet'
    limit 10
""")
res.show()


┌─────────┐
│ max(fs) │
│  int64  │
├─────────┤
│  279663 │
└─────────┘



In [ ]:
# Schema is (fs (table_id) int, i (row_id) int, j (column_id) int, v (value) int)
# Task 1: Find table pairs with many values in common.
res = duckdb.sql("""
with sdata as (
    select * from 'experiments/results/parquet_tall/df_*.parquet'
),
values as (
    select distinct fs, v from sdata
)
select v1.fs as fs1, v2.fs as fs2, count(*) as c
from values v1
join values v2
on v1.v = v2.v
where v1.fs != v2.fs
group by v1.fs, v2.fs
limit 10
""")
res.show()